In [24]:
# RQA -- in between - raw quality assessment (amplitude * power frequency ratio * alpha band psd ratio)
# Extract file ---> collective data ---> Bad channel removal ----- Not needed [sampling rate correction (500Hz)] -----
# filtering (butterworth bandpass - tune coefficient)  ----- rereferencing ----- epoching ----- ERP ----- baseline removal ---- 

In [25]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import fftpack,  signal
import time
import mne
import matplotlib
from collections import UserDict


## Reading the eeg data

In [26]:
channel_str='Channel 1:P8\
	Channel 2:T8\
	Channel 3:CP6\
	Channel 4:FC6\
	Channel 5:F8\
	Channel 6:F4\
	Channel 7:C4\
	Channel 8:P4\
	Channel 9:AF4\
	Channel 10:Fp2\
	Channel 11:Fp1\
	Channel 12:AF3\
	Channel 13:Fz\
	Channel 14:FC2\
	Channel 15:Cz\
	Channel 16:CP2\
	Channel 17:PO3\
	Channel 18:O1\
	Channel 19:Oz\
	Channel 20:O2\
	Channel 21:PO4\
	Channel 22:Pz\
	Channel 23:CP1\
	Channel 24:FC1\
	Channel 25:P3\
	Channel 26:C3\
	Channel 27:F3\
	Channel 28:F7\
	Channel 29:FC5\
	Channel 30:CP5\
	Channel 31:T7\
	Channel 32:P7'

string_channels = channel_str.replace('\t', ':').split(':')
channel_names = [string_channels[i] for i in range(len(string_channels)) if i % 2 != 0]
channel_names.append('ax')
channel_names.append('ay')
channel_names.append('az')
channel_names.append('trigger')
channel_names.append('timestamp(ms)')

In [27]:
import pandas as pd

folder_path = 'E:/IITD/Depression-IITD/Depression-Sample-dataset-AIIMS/'
sham_or_active = 'Active/' # Active
patient = 'Preeti singh/'
pre_post_intervention = 'pre/'
directory = folder_path + sham_or_active + patient + pre_post_intervention
file_path = directory + '20230718202514_Preeti singh_22.08.23-01_GNG.easy'

df = pd.read_csv(file_path, sep='\t')
df.columns=channel_names
df.head()

,P8,T8,CP6,FC6,F8,F4,C4,P4,AF4,Fp2,...,F7,FC5,CP5,T7,P7,ax,ay,az,trigger,timestamp(ms)
0,31399486,41813101,37317782,43707527,3615337,26556925,36763121,21657737,9144579,20491800,...,20403151,24447225,23063074,20909999,38154017,-490,-9865,-68,0,1689692114103
1,31407662,41821620,37328919,43716121,3626461,26563954,36772044,21666019,9151566,20499466,...,20413408,24458780,23074804,20922896,38166180,-107,-9747,225,0,1689692114105
2,31414594,41827732,37333736,43719642,3632327,26566235,36775766,21669698,9148165,20499951,...,20418334,24465610,23081071,20930846,38175052,-304,-9865,-29,0,1689692114107
3,31417887,41829310,37334285,43720062,3633515,26566595,36776069,21672006,9144064,20498742,...,20418375,24465020,23084448,20931321,38178422,-264,-9904,68,0,1689692114109
4,31422884,41828356,37333614,43717552,3630525,26564635,36775617,21674890,9146353,20502239,...,20418384,24464380,23088964,20932807,38181743,-411,-9708,-68,0,1689692114111


## MNE Analysis

In [28]:
transposed_data=df.T

# Create a MNE-Python info object and specifying sampling rate of data
ch_names = df.columns.tolist()[:-5]
ch_types = ['eeg' for i in range(32)]
info = mne.create_info(ch_names=ch_names,ch_types=ch_types, sfreq=500)

# Create a Raw object from the DataFrame
raw = mne.io.RawArray(transposed_data.values[:-5,:], info)

Creating RawArray with float64 data, n_channels=32, n_times=149999
    Range : 0 ... 149998 =      0.000 ...   299.996 secs
Ready.


In [29]:
# Printing information about the number of channels and the shape of the data
print(f"num of channels: {raw.info.get('nchan')}")
print(f'Shape of the data: {raw.get_data().shape}')

num of channels: 32
Shape of the data: (32, 149999)


### 1. Time amplitude plot

In [30]:
def time_amplitude(title):
    # To plot raw eeg plot
    plt.switch_backend('TkAgg')

    # Bandpass butterworth filter added
    # Cutoffs at 0.01, 45.00 Hz: -6.02, -6.02 dB
    # Applied only in currently plot
    fig = raw.plot(n_channels=32, 
    lowpass=45, 
    highpass=0.01,
    scalings={'eeg': 1e5} 
    )

    # plt.title(f'{title} - Time-Amplitude - Channel {map[i+1]}')
    fig.savefig(f'MNE-graphs/time-amplitude/{title}-EEG.png')

    # In interactive plot, mark bad channels and they will be assigned to 'bads'
    print(raw.info)

### 2. Power spectral density plot

In [31]:
def psd(title):
    # To plot power spectral density plot
    fig = raw.compute_psd(fmax=150,picks=np.array(channel_names[:-5])).plot(picks=np.array(channel_names[:-5]), exclude="bads")
    fig.savefig(f'MNE-graphs/psd-frequency/{title}.png')

In [32]:
# Can plot graph before preprocessing
title = "0--Raw graph with line filter"
time_amplitude(title)
psd(title)

Setting up band-pass filter from 0.01 - 45 Hz

IIR filter parameters
---------------------
Butterworth bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backward)
- Cutoffs at 0.01, 45.00 Hz: -6.02, -6.02 dB

<Info | 7 non-empty values
 bads: []
 ch_names: P8, T8, CP6, FC6, F8, F4, C4, P4, AF4, Fp2, Fp1, AF3, Fz, FC2, ...
 chs: 32 EEG
 custom_ref_applied: False
 highpass: 0.0 Hz
 lowpass: 250.0 Hz
 meas_date: unspecified
 nchan: 32
 projs: []
 sfreq: 500.0 Hz
>
Effective window size : 4.096 (s)


C:\Users\vishw\AppData\Local\Temp\ipykernel_29004\2242135612.py:3: RuntimeWarning: Channel locations not available. Disabling spatial colors.
  fig = raw.compute_psd(fmax=150,picks=np.array(channel_names[:-5])).plot(picks=np.array(channel_names[:-5]), exclude="bads")


### Preprocessing

1. Bad channels removal

Interpolation required to estimate missing or bad channel values.

In [33]:
# raw.info['bads'] = # can set manually through interactive plot
# TODO: Code to automatically mark channels bad
raw = raw.interpolate_bads(reset_bads=False)
# title = "1--Bad channels removed"
# time_amplitude(title)
# psd(title)

C:\Users\vishw\AppData\Local\Temp\ipykernel_29004\2423377182.py:3: RuntimeWarning: No bad channels to interpolate. Doing nothing...
  raw = raw.interpolate_bads(reset_bads=False)


2. Band pass filtering

In [34]:
# applied band pass filter of 0.01-45 Hz for depression detection 
raw.filter(0.01, 45, fir_design='firwin', picks=['eeg'])
title = "2--Band pass filtered graph"
# time_amplitude(title)
psd(title)

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.01 - 45 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.01
- Lower transition bandwidth: 0.01 Hz (-6 dB cutoff frequency: 0.01 Hz)
- Upper passband edge: 45.00 Hz
- Upper transition bandwidth: 11.25 Hz (-6 dB cutoff frequency: 50.62 Hz)
- Filter length: 165001 samples (330.002 s)



C:\Users\vishw\AppData\Local\Temp\ipykernel_29004\2505533690.py:2: RuntimeWarning: filter_length (165001) is longer than the signal (149999), distortion is likely. Reduce filter length or filter a longer signal.
  raw.filter(0.01, 45, fir_design='firwin', picks=['eeg'])
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.5s


Effective window size : 4.096 (s)


C:\Users\vishw\AppData\Local\Temp\ipykernel_29004\2242135612.py:3: RuntimeWarning: Channel locations not available. Disabling spatial colors.
  fig = raw.compute_psd(fmax=150,picks=np.array(channel_names[:-5])).plot(picks=np.array(channel_names[:-5]), exclude="bads")


3. Rereferencing

Calculates the mean voltage from all electrodes at each time point and subtracts this mean from the voltage at each individual electrode.

In [35]:
raw.set_eeg_reference('average', projection=True).apply_proj() 
title = "3---Rereferenced"
time_amplitude(title)
psd(title)

EEG channel type selected for re-referencing
Adding average EEG reference projection.
1 projection items deactivated
Average reference projection was added, but has not been applied yet. Use the apply_proj method to apply it.
Created an SSP operator (subspace dimension = 1)
1 projection items activated
SSP projectors applied...
Setting up band-pass filter from 0.01 - 45 Hz

IIR filter parameters
---------------------
Butterworth bandpass zero-phase (two-pass forward and reverse) non-causal filter:
- Filter order 16 (effective, after forward-backward)
- Cutoffs at 0.01, 45.00 Hz: -6.02, -6.02 dB

<Info | 8 non-empty values
 bads: []
 ch_names: P8, T8, CP6, FC6, F8, F4, C4, P4, AF4, Fp2, Fp1, AF3, Fz, FC2, ...
 chs: 32 EEG
 custom_ref_applied: False
 highpass: 0.0 Hz
 lowpass: 45.0 Hz
 meas_date: unspecified
 nchan: 32
 projs: Average EEG reference: on
 sfreq: 500.0 Hz
>
Effective window size : 4.096 (s)


C:\Users\vishw\AppData\Local\Temp\ipykernel_29004\2242135612.py:3: RuntimeWarning: Channel locations not available. Disabling spatial colors.
  fig = raw.compute_psd(fmax=150,picks=np.array(channel_names[:-5])).plot(picks=np.array(channel_names[:-5]), exclude="bads")


4. Deleting annotations - Not for first iteration


In [36]:
visual_inspection = pd.read_csv(directory + "Preeti singh_GoNogo_Test_2023_Jul_18_2026.csv")
events_from_annot, event_dict = mne.events_from_annotations(raw)

for i in range(visual_inspection.shape[0]):         
    result = np.where((mne.events_from_annotations(raw)[0][:,0] > visual_inspection['Response.started'][i]*500) & 
                      (mne.events_from_annotations(raw)[0][:,0] < visual_inspection['Response.stopped'][i]*500))
    raw.annotations.delete(result)

#### 5. Epoching

5.1 Adding trigger channel

In [37]:
# events_from_annot, event_dict = mne.events_from_annotations(raw)# Get events and event_id from an Annotations object.
trigger_col = df.values[:, -2]
trigger_times = []
event_id = {}
event_list = []

for i in range(len(trigger_col)):
    if trigger_col[i]!=0:
        trigger_times.append(i+2)
        event_id[trigger_col[i]] = trigger_col[i]
        event_list.append(trigger_col[i])

# Create an events array (trigger value, previous, sample number)
events = np.column_stack((trigger_times, np.zeros_like(trigger_times), event_list))

fig = raw.plot(n_channels=32, events=events, event_id=event_id, event_color={1:'r', 2:'g', 3:'b'}, scalings={'eeg': 1e5})
fig.savefig(f'MNE-graphs/time-amplitude/4---Trigger channels added.png')

5.2 Defining and segmenting Epochs

In [57]:
# Applying baseline correction (mode: mean) - baseline=(tmin, 0)
# Created an SSP operator (subspace dimension = 1)
tmin = -0.01
tmax = 0.05

# reject_criteria = dict(eeg=100e-6) # Absolute Amplitude of each epoch sould be smaller than 100 μV
# Add reject_criteria=reject_criteria in mne.Epochs
epochs = mne.Epochs(raw, events=events, event_id=None, tmin=tmin, tmax=tmax, baseline=(tmin, 0), detrend=1, preload=True,picks=['eeg'])
fig = epochs.plot(n_channels=32, scalings={'eeg': 1e5}, event_color={1:'r', 2:'g', 3:'b'}, events=events)
fig.savefig(f'MNE-graphs/time-amplitude/5---Epochs added.png')

print(epochs)

Not setting metadata
146 matching events found
Applying baseline correction (mode: mean)
Created an SSP operator (subspace dimension = 1)
1 projection items activated
Using data from preloaded Raw for 146 events and 31 original time points ...
0 bad epochs dropped
You seem to have overlapping epochs. Some event lines may be duplicated in the plot.
<Epochs |  146 events (all good), -0.01 – 0.05 s, baseline -0.01 – 0 s, ~1.1 MB, data loaded,
 '1': 99
 '2': 24
 '3': 23>


Dropped 0 epochs: 
The following epochs were marked as bad and are dropped:
[]
Channels marked as bad:
['T8']


6. Artifact Rejection (EOG/ECG) using ICA

In [61]:
n_components = 31
ica = mne.preprocessing.ICA(n_components=n_components, random_state=97, method='fastica', max_iter=100)
ica.fit(epochs)

Fitting ICA to data using 31 channels (please be patient, this may take a while)
    Applying projection operator with 1 vector (pre-whitener computation)
    Applying projection operator with 1 vector (pre-whitener application)
Selecting by number: 31 components


C:\Users\vishw\AppData\Local\Temp\ipykernel_29004\2436572980.py:3: RuntimeWarning: The epochs you passed to ICA.fit() were baseline-corrected. However, we suggest to fit ICA only on data that has been high-pass filtered, but NOT baseline-corrected.
  ica.fit(epochs)


    Applying projection operator with 1 vector (pre-whitener application)
Fitting ICA took 0.3s.


C:\Users\vishw\AppData\Local\Temp\ipykernel_29004\2436572980.py:3: RuntimeWarning: Using n_components=31 (resulting in n_components_=31) may lead to an unstable mixing matrix estimation because the ratio between the largest (14) and smallest (1.3e-31) variances is too large (> 1e6); consider setting n_components=0.999999 or an integer <= 30
  ica.fit(epochs)


ModuleNotFoundError: No module named 'UserDict'

<ICA | epochs decomposition, method: fastica (fit in 68 iterations on 4526 samples), 31 ICA components (31 PCA components available), channel types: eeg, no sources marked for exclusion>

In [ ]:
%matplotlib inline
ica.plot_components()

In [ ]:
# Choose the components representing artifacts to be removed
# Use the indices of the components identified as artifacts
# TODO: Atomate this 
ica.exclude = []  # Put all comonent which you want to remove containg inspected (manual) 
ica.apply(epochs)                       